# 1 - Simplified Introduction

The aim of these notebooks is to try and introduce some Quantum Error Correction (QEC) concepts without talking about QEC at all!

A large part of many QEC protocols involves a decent portion of classical computation, which itself can be fairly complex. We will focus on these classical parts, and relate them back to quantum computing later.

The following notebook introduces a graph puzzle that is entirely a classical computational problem, but relevant to a large number of QEC encoding/decoding protocols. The problem is described without using any confusing QEC language, in a way that is accessible to anyone with a background in classical computer science.


## A classical computational graph puzzle

Consider a periodic graph defined as coordinates over edges (`E`) and vertices (`V`) that are arranged in a `width x length` grid structure like so:

In [157]:
from src.classical.periodic_grid_graph import PeriodicGridGraph

graph = PeriodicGridGraph(width=5, length=5)
graph.draw_graph()


D = 5x5 repeating graph

                                                                                               
                                                                                               
               E0              E1              E2              E3              E4              
                                                                                               
                                                                                               
       E5      V0      E6      V1      E7      V2      E8      V3      E9      V4      E5      
                                                                                               
                                                                                               
               E10             E11             E12             E13             E14             
                                                                                               
              

This example has `width = length = 5`, and indexes run from left-to-right, top-to-bottom, for both edges and vertices.

The graph has periodic boundary conditions, where the left/right columns and top/bottom rows of edges are identical. In other words, the graph "wraps around" like a pacman grid - if you travel beyond the top of the grid you will re-emerge at the bottom, and vice versa. Same goes for left/right.

Each edge connects 2 adjacent vertices, and every vertex has a degree of 4 (i.e. is incident to 4 edges).

Now we have defined the graph layout, let's introduce some rules:

- both edges and vertices can exist in one of two states: `0` or `1`;
- every edge and vertex is initialised in the `0` state;
- if an edge is in the state `1`, then the state of its 2 adjacent vertices is also `1`;
- only edges may have their state changed directly - the state of a vertex can only be changed indirectly as a result of the state change of an adjacent edge.

To illustrate these, let's take a look at the same graph again, where all edges and vertices are ininitialsed in the `0` state:

In [158]:
graph.draw_graph()


D = 5x5 repeating graph

                                                                                               
                                                                                               
               E0              E1              E2              E3              E4              
                                                                                               
                                                                                               
       E5      V0      E6      V1      E7      V2      E8      V3      E9      V4      E5      
                                                                                               
                                                                                               
               E10             E11             E12             E13             E14             
                                                                                               
              

When edges get flipped to `1`, they turn red; when vertices get flipped to `1`, they turn yellow.

Let's see what happens when we flip the state of edge `E17`:

In [159]:
edges_to_flip = [17]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)



D = 5x5 repeating graph

                                                                                               
                                                                                               
               E0              E1              E2              E3              E4              
                                                                                               
                                                                                               
       E5      V0      E6      V1      E7      V2      E8      V3      E9      V4      E5      
                                                                                               
                                                                                               
               E10             E11             E12             E13             E14             
                                                                                               
              

We see here that by flipping edge `E17` from `0` to `1` we have also indirectly flipped the state of vertices `V6` and `V7` from `0` to `1`.

What happens if we also flip edge `E22`? Let's have a look:

In [160]:
edges_to_flip = [17, 22]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


D = 5x5 repeating graph

                                                                                               
                                                                                               
               E0              E1              E2              E3              E4              
                                                                                               
                                                                                               
       E5      V0      E6      V1      E7      V2      E8      V3      E9      V4      E5      
                                                                                               
                                                                                               
               E10             E11             E12             E13             E14             
                                                                                               
              

We see here that by flipping edges `E17` and `E22` we have also flipped the state of vertices `V7` and `V12.

But since both `E17` and `E22` have each flipped the state of `V7`, that vertex goes back to the original `0` state.
This introduces the concept of the flipped edge "chain": given a chain of adjacent flipped-edges, only vertices at the ends of the chain will be flipped!

From all this, we see that <u>flipped vertices always appear in pairs</u>.

What happens when we flip the states of edges `E21` and `E27`?

In [161]:
edges_to_flip = [21, 27]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


D = 5x5 repeating graph

                                                                                               
                                                                                               
               E0              E1              E2              E3              E4              
                                                                                               
                                                                                               
       E5      V0      E6      V1      E7      V2      E8      V3      E9      V4      E5      
                                                                                               
                                                                                               
               E10             E11             E12             E13             E14             
                                                                                               
              

We see that this has resulted in flipping the exact same vertices that edges `E17` and `E22` did!

A similar thing can happen if the flipped edge chain crosses one of the graph boundaries.
Consider what happens when we flip edges `E25`, `E26` and `E27`:

In [162]:
edges_to_flip = [25, 26, 27]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


D = 5x5 repeating graph

                                                                                               
                                                                                               
               E0              E1              E2              E3              E4              
                                                                                               
                                                                                               
       E5      V0      E6      V1      E7      V2      E8      V3      E9      V4      E5      
                                                                                               
                                                                                               
               E10             E11             E12             E13             E14             
                                                                                               
              

The eagle-eyed will notice the same vertices would be flipped if edges `E28` and `E29` were flipped. In fact, there are quite a few flipped-edge chains that would do this, for example:

In [163]:
edges_to_flip = [6, 7, 8, 9, 10, 14, 16, 17, 22, 24]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


D = 5x5 repeating graph

                                                                                               
                                                                                               
               E0              E1              E2              E3              E4              
                                                                                               
                                                                                               
       E5      V0      E6      V1      E7      V2      E8      V3      E9      V4      E5      
                                                                                               
                                                                                               
               E10             E11             E12             E13             E14             
                                                                                               
              

We see here that many flipped-edge chains can be equivalent with respect to the vertices that are ultimately flipped.

Of course, edges flipped on different areas of the graph will result in even more flipped vertex pairs:

In [164]:
edges_to_flip = [8, 9, 14, 20, 24, 26]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


D = 5x5 repeating graph

                                                                                               
                                                                                               
               E0              E1              E2              E3              E4              
                                                                                               
                                                                                               
       E5      V0      E6      V1      E7      V2      E8      V3      E9      V4      E5      
                                                                                               
                                                                                               
               E10             E11             E12             E13             E14             
                                                                                               
              

What happens when an edge chain loops back on itself? This could happen two ways:
- going from end-to-end either top/bottom or left/right;
- forming a closed loop.

Lets see what happens in these cases:

In [165]:
edges_to_flip = [1, 7, 12, 17, 21, 31, 41, 29, 33, 34, 39]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


D = 5x5 repeating graph

                                                                                               
                                                                                               
               E0              E1              E2              E3              E4              
                                                                                               
                                                                                               
       E5      V0      E6      V1      E7      V2      E8      V3      E9      V4      E5      
                                                                                               
                                                                                               
               E10             E11             E12             E13             E14             
                                                                                               
              

No vertices are in the `1` state! For now this isn't too important, but is interesting to note.


## The problem to solve

Now we have defined the structure of the problem, an interesting question arises: given a grid which has some of its vertices flipped, can you determine the shortest edge chains that connect pairs of the flipped vertices?

In other words, how do we set the state of all flipped vertices back to `0` by flipping the smallest number of edges?

For example, consider this grid:

In [166]:
graph = PeriodicGridGraph(6, 6)
flipped_vertices = [3, 12, 15, 19]
graph.draw_graph(vertices=flipped_vertices)


D = 6x6 repeating graph

                                                                                                               
                                                                                                               
               E0              E1              E2              E3              E4              E5              
                                                                                                               
                                                                                                               
       E6      V0      E7      V1      E8      V2      E9      V3      E10     V4      E11     V5      E6      
                                                                                                               
                                                                                                               
               E12             E13             E14             E15            

Here, the shortest path between vertices `V3` and `V15` is via edges `E15` and `E27`, so flipping these edges will reset `V3` and `V15` back to `0`.

For vertices `V12` and `V19` there are two equivalent paths: via edges `E31` and `E37` or edges `E36` and `E43`.

There are other paths that connect different vertex pairs, but none of these are shorter than the ones described already.

What if we add a few more vertices?

In [167]:
flipped_vertices = [0, 3, 5, 12, 15, 19]
graph.draw_graph(vertices=flipped_vertices)


D = 6x6 repeating graph

                                                                                                               
                                                                                                               
               E0              E1              E2              E3              E4              E5              
                                                                                                               
                                                                                                               
       E6      V0      E7      V1      E8      V2      E9      V3      E10     V4      E11     V5      E6      
                                                                                                               
                                                                                                               
               E12             E13             E14             E15            

It's tempting to say that vertices `V3` and `V5` could be paired up via edges `E10` and `E11`, leaving `V15` to be paired with some other vertex.

But if we recall the periodic boundary conditions then we see that `V5` can actually be flipped by a shorter path to `V0` via edge `E6`:

In [168]:
flipped_vertices = [0, 3, 5, 12, 15, 19]
flipped_edges = [6, 15, 27, 36, 43]
graph.draw_graph(vertices=flipped_vertices, edges=flipped_edges)


D = 6x6 repeating graph

                                                                                                               
                                                                                                               
               E0              E1              E2              E3              E4              E5              
                                                                                                               
                                                                                                               
       E6      V0      E7      V1      E8      V2      E9      V3      E10     V4      E11     V5      E6      
                                                                                                               
                                                                                                               
               E12             E13             E14             E15            

So, can we come up with an algorithm that helps solves this puzzle?